# Clinical trials in R. Example
### Compare the risks of two drugs
**Task.** A prospective [study](https://www.nejm.org/doi/full/10.1056/NEJM200005183422004) compares the incidences of dyskinesia after ropinirole (ROP) or levodopa (LD) in patients with early Parkinson’s disease. The results show that 16 of 179 patients who took ropinirole and 23 of 89 who took levodopa developed dyskinesia. The data is summarized in Table below. Which treatment is better in avoiding the unwanted effect of dyskinesia?

Drug|Dyskinesia present|Dyskinesia absent|Total
-|-|-|-
LD|23|66|89
ROP|16|163|179
**Total**|39|229|

**Notes:**
1. In this solution all confidence intervals wherever it's possible will be calculated as suggested in this [article](https://doi.org/10.1046/J.1524-4733.2002.55150.x)
2. All interpretations will be done as described in this [book](https://www.ncbi.nlm.nih.gov/books/NBK63647/)

### 1.	Choose group to act as a baseline risk provider
Because the study wasn't designed to have a dedicated *control group* we have to designate one of the groups to be a baseline risk provider, which from the calculation point of view will act as an effective control group to make comparisons against. Let *LD* group act as a Baseline risk provider (BRP), then ROP group will fill in the role of a *Treatment group (TG)* relative to ROP group:

+ $R_{BRP} = R_{LD}$
+ $R_{TG} = R_{ROP}$

**Get data in R**

Let's use the existing abbreviatons of the drugs and introduce new ones: *DP* for dyskinesia present, *DA* for dyskinesia absent.

In [1]:
Input = (
"Drug,LD,ROP
DP,23,16
DA,66,163"
) 
(Data = read.csv(textConnection(Input), header = TRUE, row.names = 1, strip.white = TRUE))

,LD,ROP
DP,23,16
DA,66,163


### 2.	Calculate the *Risk* for each group
To calculate the *Risks* one has to identify the *Event* and *NonEvent* outcomes. From the evidence-based medicine perspective *Event* is any kind of negative (in a common sense) outcome.

Outcome types:
+ *Event* - Dyskinesia present
+ *NonEvent* - Dyskinesia absent

By definition *Risk (R)* is probability of *Event*: 
$$R = \frac{Event}{Event + NonEvent}$$

In [2]:
Risk = list(LD = Data$LD[1] / sum(Data$LD), 
            ROP = Data$ROP[1] / sum(Data$ROP))

**Confidence intervals of the Risks** one can determine through the binomial test just like any nominal data

In [3]:
Risk$LD.CI = binom.test(Data$LD[1], sum(Data$LD))$conf.int
Risk$ROP.CI = binom.test(Data$ROP[1], sum(Data$ROP))$conf.int
# Collect all calculations in one variable
Risk = data.frame(row.names = c("Risk", "lwr.CI", "upr.CI"), 
                  cbind(ROP = c(Risk$ROP, Risk$ROP.CI), 
                        LD = c(Risk$LD, Risk$LD.CI))
                 )

In [4]:
# Show Risks and their CI
round(Risk, digits = 3)

,ROP,LD
Risk,0.089,0.258
lwr.CI,0.052,0.171
upr.CI,0.141,0.362


### 3. Calculate the *Absolute Risk Reduction* (ARR)
ARR is a difference between baseline risk provider's (e.g. control group) value of risk and treatment group risk:

$$ARR = R_{BRP} - R_{TG}$$

**Confidence intervals for ARR** will be obtained with `BinomDiffCI` function from `DescTools` library. This function implements variety of methods to obtain values for ARR and it's confidence intervals. We will conjure it with argument `method = "score` to specify [Newcombe](https://doi.org/10.1002/(SICI)1097-0258(19980430)17:8%3C873::AID-SIM779%3E3.0.CO;2-I) method.

In [5]:
library(DescTools)

Warning message:
"package 'DescTools' was built under R version 3.6.3"

In [6]:
ARR = BinomDiffCI(x1 = Data$LD[1], n1 = sum(Data$LD), x2 = Data$ROP[1], n2 = sum(Data$ROP), method = "score")
# Show ARR and it's CI
round(ARR, digits = 3)

est,lwr.ci,upr.ci
0.169,0.075,0.274


**Interpretation:** Absolute risk reduction (ARR) is 16.9% with it's confidence interval of 7.5% to 27.4%. Which means that, ropinirole treatment at average allows almost 17 out of 100 patients not to develop dyskinesia as opposed to the levadopa treatment.

### 4. Calculate the *Relative Risk* (RR)
RR is a ratio of risks of the treated group and the control group:
$$RR = \frac{R_{TG}}{R_{BRP}}$$

**Confidence intervals for RR** will be calculated with `BinomRatioCI` function from `DescTools` library. This function implements various methods for calculations of the confidence intervals for the ratio of two binomial proportions. We will call this function without specifying argument `method =` which will force function to use the [Katz-log](https://doi.org/10.2307/2530610) method.

In [7]:
RR = BinomRatioCI(x1 = Data$ROP[1], n1 = sum(Data$ROP), x2 = Data$LD[1], n2 = sum(Data$LD))
# Show RR and it's CI
round(RR, digits = 3)

est,lwr.ci,upr.ci
0.346,0.193,0.621


**Interpretation:** Relative risk of ROP and LD treatment is less than 1, which means risk of developing dyskinesia during ropinirole treatment is lower than the risk of developing dyskinesia during levadopa treatment. At average risk of developing dyskinesia is $\approx$2.9 times lower with ROP than with LD. Obtained result is consistent with the ARR results.

### 5. Calculatte the *Relative Risk Reduction* (RRR)
$$RRR = 1 - RR$$

In [8]:
RRR = 1 - RR
# Show RRR and it's CI
round(RRR, digits = 3)

est,lwr.ci,upr.ci
0.654,0.807,0.379


**Interpretation:** Relative risk reduction value shows that ropinirole treatment allows to decrease the risk of developing dyskinesia at average on 65.4% relative to the levadopa treatment.

### 6. Calculate the *Odds Ratio* (OR)
Odd is a ratio between probabilities of Event and NonEvent and can be expressed as:
$$O = \frac{Event}{NonEvent}$$

Odds ratio is a ratio between the odds of the treatment group and baseline risk provider group:
$$OR = \frac{O_{TG}}{O_{BRP}}$$

**Confidence intervals for OR** will be calculated with `BinomRatioCI` function from `DescTools` library. As in case of RR confidence intervals we'll use the Katz-log method.

In [9]:
OR = BinomRatioCI(x1 = Data$ROP[1], n1 = Data$ROP[2], x2 = Data$LD[1], n2 = Data$LD[2])
# Show OR and it's CI
round(OR, digits = 3)

est,lwr.ci,upr.ci
0.282,0.159,0.498


**Interpretation:** Confidence interval for Odds ratio is 0.16 to 0.5 with the average value of 0.282, which means  dyskinesia development at average in 28.2% of ROP treatment cases of LD dyskinesia development cases. Being less than 1, OR shows risk reduction in ROP treatment group relative to LD treatment group.

### 7. Calculate the *Number needed to treat* (NNT)
NNT is a measure tightly related to the ARR and is an inverse of ARR value:
$$NNT = \frac{1}{ARR}$$

**Confidence intervals of NNT** can be obtained via simple inversion of the confidence interval bounds of ARR and so have the same formula as NNT point estimate.

In [10]:
NNT = 1 / ARR
# Show NNT and it's CI
round(NNT, digits = 1)

est,lwr.ci,upr.ci
5.9,13.4,3.6


**Interpretation:** NNT belong to the range of values beetween 3.6 and 13.4 with an average of 5.9. This means that to get one additional patient without developing dyskinesia relative to LD treatment one has to expose $\approx$6 patients  to ROP treatment.

### Conclusion:
*Ropinirole* treatment has reduced risk of developing dyskinesia in comparison to *levadopa* treatment and should be considered the better option among the two.